Working with MNIST dataset again
Importing nessesary libraries first

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np

Importing MNIST

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Checking data dimensionality

In [ ]:
print("Training data shape: ", x_train.shape) # (60000, 28, 28) -- 60000 images, each 28x28 pixels
print("Test data shape", x_test.shape) # (10000, 28, 28) -- 10000 images, each 28x28
print("Training response shape:, ", y_train.shape)
print("Testing response shape: ", y_test.shape)

image_size = (x_train.shape[1], x_train.shape[2])

Need to flatten the images to work with MLP

In [ ]:
# Flatten the images
image_vector_size = image_size[0] * image_size[1] # 28 * 28
x_train = x_train.reshape(x_train.shape[0], image_vector_size) /255.
x_test = x_test.reshape(x_test.shape[0], image_vector_size) /255.
print(x_train.shape)

We are going to corrupt MNIST images with noise and train a denoising autoencoder to reconstruct the original images

In [ ]:
noise_factor = 0.7 #determines how much images are corrupted
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

Let's plot one such image with noise

In [ ]:
import matplotlib.pyplot as plt
i=5
#n = 10
#plt.figure(figsize=(20, 2))
#for i in range(n):
#    ax = plt.subplot(1, n, i)
plt.imshow(x_test_noisy[i].reshape(28, 28))
plt.gray()
#    ax.get_xaxis().set_visible(False)
#    ax.get_yaxis().set_visible(False)
plt.show()

Below we define an autoencoder model with the dimension of bottleneck layer of 32

In [ ]:
# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

We can also separetely access encoder and decoder parts of this model[link text](https://)

In [ ]:
encoder = Model(input_img, encoded)

In [ ]:
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

Let's compile our autoencoder with Adadelta optimiser

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

Let's fit the model for 50 epoch, notice that we don't use Labels at all anymore!

In [ ]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Let's check out visually how well our autoencoder is doing. We will visualise the results of our autoencoder applied to test set of noisy images

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test_noisy)
decoded_imgs = decoder.predict(encoded_imgs)

We will visualise the original images in the top row
and the reconstructed images in the bottow row

In [ ]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Yay!!! It did the job quite well!! Try to vary noise parameter above.
But now let's apply this autoencoder to noisy version of the Fashion-MNIST dataset

In [ ]:
from keras.datasets import fashion_mnist

(Fx_train, Fy_train), (Fx_test, Fy_test) = fashion_mnist.load_data()

In [ ]:
print("Training data shape: ", x_train.shape) # (60000, 28, 28) -- 60000 images, each 28x28 pixels
print("Test data shape", x_test.shape) # (10000, 28, 28) -- 10000 images, each 28x28
print("Training response shape:, ", y_train.shape)
print("Testing response shape: ", y_test.shape)

Fimage_size = (Fx_train.shape[1], Fx_train.shape[2])

In [ ]:
# Flatten the images
Fimage_vector_size = Fimage_size[0] * Fimage_size[1] # 28 * 28
Fx_train = Fx_train.reshape(Fx_train.shape[0], Fimage_vector_size) /255.
Fx_test = Fx_test.reshape(Fx_test.shape[0], Fimage_vector_size) /255.
print(Fx_train.shape)

We applied noise to the Fashion-MNIST images below

In [ ]:
noise_factor = 0.7
Fx_train_noisy = Fx_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=Fx_train.shape) 
Fx_test_noisy = Fx_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=Fx_test.shape) 

Let's now apply previously trained autoencoder to reconstruct them back

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
Fencoded_imgs = encoder.predict(Fx_test_noisy)
Fdecoded_imgs = decoder.predict(Fencoded_imgs)

In [ ]:
Fdecoded_imgs =autoencoder.predict(Fx_test_noisy)

Woah! So close. Let's visuale the results!

In [ ]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 20  # how many digits we will display
plt.figure(figsize=(40, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(Fx_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(Fdecoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Are you suprised? =) Can you explain what happened. 
Let's apply autoencoder to these weird looking symbols again

In [ ]:
FMdecoded_imgs=autoencoder.predict(Fdecoded_imgs)

In [ ]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 20  # how many digits we will display
plt.figure(figsize=(40, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(Fdecoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(FMdecoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Try do another way around - train an autoencoder on Fashion-MNIST and apply it to MNIST